<a href="https://colab.research.google.com/github/dgg32/db_neo4j/blob/master/kepler_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install keplergl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.7 MB 7.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.0 MB 66.0 MB/s 
     |████████████████████████████████| 6.3 MB 48.2 MB/s 
     |████████████████████████████████| 16.7 MB 63.1 MB/s 
  Using cached jedi-0.18.1-py2.py3-none-any.whl (1.6 MB)
  Created wheel for keplergl: filename=keplergl-0.3.2-py2.py3-none-any.whl size=17922344 sha256=f48bcfd92e309dca379b00e7bad8e48aead3e6a14b395b6bad32b52face20745
  Stored in directory: /root/.cache/pip/wheels/4b/19/e5/0852b7888ee350e9c57087ccc61eeab9daf50f5caac9c690ec
Successfully built keplergl


In [13]:
from keplergl import KeplerGl
import pandas as pd
import requests
from google.colab import output
output.enable_custom_widget_manager()

In [5]:
bahnhof = pd.read_csv("data/bahnhof.csv")
bahnhof.head()

,EVA_NR,DS100,NAME,Verkehr,longitude,latitude,state
0,8001944,TETN,Eutingen Nord,RV,8.753100,48.484700,Baden-Württemberg
1,8003074,MIA,Ingolstadt Audi,RV,11.407456,48.790496,Bayern
2,8001510,TDSA,Dornstetten-Aach,RV,8.482910,48.473300,Baden-Württemberg
3,8001966,MFOL,Feldolling,nur DPN,11.852244,47.895336,Bayern
4,8002535,EOBG,Halver-Oberbrügge,RV,7.574042,51.191867,Nordrhein-Westfalen


In [6]:
bahnhof[bahnhof["NAME"] == "Hamburg Hbf"]["DS100"].values[0]

'AH'

In [23]:
def search_route(start, end, time, time_type):
    #curl -X POST -H "Accept: application/json" -H "Content-Type: application/json" 
    #"https://openapi.trassenfinder.de/3.12/api/v5/routen/suche"  
    # -d '{"infrastruktur_id": 10, "sucheinstellungen": {"an_abzeit": "2022-11-13T16:38:36+02:00", "verkehrsart": "spfv_lok", "zeitvorgabe_typ": "abzeit"},  
    # "wegpunkte": [{"betriebsstelle": {"ds100": "HBS"}}, {"betriebsstelle": {"ds100":"MH"}}]}'
    start_ds100 = bahnhof[bahnhof["NAME"] == start]["DS100"].values[0]
    end_ds100 = bahnhof[bahnhof["NAME"] == end]["DS100"].values[0]

    headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}
    data = {"infrastruktur_id": 10, "sucheinstellungen": {"an_abzeit": time, "verkehrsart": "spfv_lok", "zeitvorgabe_typ": time_type},  
    "wegpunkte": [{"betriebsstelle": {"ds100": start_ds100}}, {"betriebsstelle": {"ds100":end_ds100}}]}

    #print (data)

    r = requests.post('https://openapi.trassenfinder.de/3.12/api/v5/routen/suche', headers=headers, json=data)

    return r.json()

In [24]:
route = search_route("Braunschweig Hbf", "München Hbf", "2022-11-04T17:48:36+02:00", "abzeit")

In [28]:
route["result"]["gewichtete_route"]["routenpunkte"][:5]

[{'ds100': 'HBS',
  'wegpunkt_index': 0,
  'naechstes_streckensegment': {'von': 'HBS',
   'bis': 'HGAA',
   'streckennummer': 1730},
  'laufende_hm': 0,
  'technische_fahrzeit_info': {'ankunft_min': 0, 'abfahrt_min': 0},
  'geschwindigkeit_technisch_hmh': 929,
  'haltart': 'kundenhalt',
  'halteplatz_sprungart': 'durchbindung',
  'schiebelok_kupplungsart': 'nichts',
  'verkehrshalt_trotz_fehlendem_bahnsteig': False,
  'halteplatz_zu_kurz': False,
  'gleiswechselbetrieb': False,
  'begegnungsverbot': False,
  'energieverbrauch_info': {'energieverbrauch_traktion_kwh': 0,
   'energieverbrauch_hilfsbetriebe_kwh': 0,
   'energieverbrauch_wagen_kwh': 0,
   'energieverbrauch_gesamt_kwh': 0},
  'energieverbrauch_kosten_euro': 0,
  'marktsegmente': [],
  'trassenpreis_euro': 0,
  'stationspreis_euro': 56,
  'kosten_fahrzeuge_personal_euro': 0,
  'nachschiebekosten_euro': 0,
  'strecke_info': {'ausserhalb_db_netz': False,
   'ungeprueft_kv': False,
   'ungeprueft_streckenklasse': False,
   'unge

In [72]:
source_lat = -1000
source_lng = -1000

target_lat = -1000
target_lng = -1000

cumulative_time = 0

cumulative_energy = 0

cumulative_distance = 0

content = ",".join(["source_lat", "source_lng", "target_lat", "target_lng", "cumulative_distance_km", "cumulative_energy_kwh", "cumulative_min"]) + "\n"

#route["result"]["gewichtete_route"]["routenpunkte"]
for s in route["result"]["gewichtete_route"]["routenpunkte"]:
    ds100 = s["ds100"]
    #print (s["technische_fahrzeit_info"])
    if source_lat == -1000:
        try:
            source_lat = bahnhof[bahnhof["DS100"] == ds100]["latitude"].values[0]
            source_lng = bahnhof[bahnhof["DS100"] == ds100]["longitude"].values[0]
            #distance = s["laufende_hm"] - cumulative_distance
            cumulative_distance = s["laufende_hm"]/10

            #energy = s["energieverbrauch_info"]["energieverbrauch_gesamt_kwh"] - cumulative_energy
            cumulative_energy = s["energieverbrauch_info"]["energieverbrauch_gesamt_kwh"]

            cumulative_time = s["technische_fahrzeit_info"]["ankunft_min"]

            content += f"{source_lat},{source_lng},{source_lat},{source_lng},{cumulative_distance},{cumulative_energy},{cumulative_time}\n"

        except:
            pass
    else:
        try:


            target_lat = bahnhof[bahnhof["DS100"] == ds100]["latitude"].values[0]
            target_lng = bahnhof[bahnhof["DS100"] == ds100]["longitude"].values[0]
            

            #distance = s["laufende_hm"] - cumulative_distance
            cumulative_distance = s["laufende_hm"]/10

            #energy = s["energieverbrauch_info"]["energieverbrauch_gesamt_kwh"] - cumulative_energy
            cumulative_energy = s["energieverbrauch_info"]["energieverbrauch_gesamt_kwh"]

            cumulative_time = s["technische_fahrzeit_info"]["ankunft_min"]
            content += f"{source_lat},{source_lng},{target_lat},{target_lng},{cumulative_distance},{cumulative_energy},{cumulative_time}\n"
            source_lat = target_lat
            source_lng = target_lng
        except:
            pass

with open("data/data_route.csv", "w") as output:
    output.write(content)

In [73]:
route_df = pd.read_csv("data/data_route.csv")
route_df.head()

,source_lat,source_lng,target_lat,target_lng,cumulative_distance_km,cumulative_energy_kwh,cumulative_min
0,52.252218,10.540295,52.252218,10.540295,0.0,0,0
1,52.252218,10.540295,52.257212,10.359760,15.0,166,8
2,52.257212,10.359760,52.318942,10.232145,25.6,217,13
3,52.318942,10.232145,52.335731,10.167325,30.4,245,15
4,52.335731,10.167325,52.354704,10.110601,35.0,273,18


In [74]:
route_df["between_distance_km"] = route_df["cumulative_distance_km"].diff().round(2)
route_df["between_energy_kwh"] = route_df["cumulative_energy_kwh"].diff()
route_df["between_min"] = route_df["cumulative_min"].diff()
route_df["between_distance_km"].fillna(0, inplace=True)
route_df["between_energy_kwh"].fillna(0, inplace=True)
route_df["between_min"].fillna(0, inplace=True)

In [75]:
route_df.head()

,source_lat,source_lng,target_lat,target_lng,cumulative_distance_km,cumulative_energy_kwh,cumulative_min,between_distance_km,between_energy_kwh,between_min
0,52.252218,10.540295,52.252218,10.540295,0.0,0,0,0.0,0.0,0.0
1,52.252218,10.540295,52.257212,10.359760,15.0,166,8,15.0,166.0,8.0
2,52.257212,10.359760,52.318942,10.232145,25.6,217,13,10.6,51.0,5.0
3,52.318942,10.232145,52.335731,10.167325,30.4,245,15,4.8,28.0,2.0
4,52.335731,10.167325,52.354704,10.110601,35.0,273,18,4.6,28.0,3.0


In [76]:
config = {'version': 'v1',
 'config': {'visState': {
   'layers': [{
     'type': 'point',
     'config': {'dataId': 'bahnhof',
      'label': 'Point',
      'color': [18, 92, 119],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'lat': 'latitude', 'lng': 'longitude', 'altitude': None},
      'isVisible': True,
      'visConfig': {'radius': 10,
       'fixedRadius': False,
       'opacity': 0.09,
       'outline': False,
       'thickness': 2,
       'strokeColor': None,
       'colorRange': {'name': 'Uber Viz Qualitative 4',
        'type': 'qualitative',
        'category': 'Uber',
        'colors': ['#12939A',
         '#DDB27C',
         '#88572C',
         '#FF991F',
         '#F15C17',
         '#223F9A',
         '#DA70BF',
         '#125C77',
         '#4DC19C',
         '#776E57',
         '#17B8BE',
         '#F6D18A',
         '#B7885E',
         '#FFCB99',
         '#F89570',
         '#829AE3',
         '#E79FD5',
         '#1E96BE',
         '#89DAC1',
         '#B3AD9E']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radiusRange': [0, 50],
       'filled': True},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': {'name': 'state', 'type': 'string'},
      'colorScale': 'ordinal',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear'}},
    {
     'type': 'arc',
     'config': {'dataId': 'route',
      'label': 'source -> target arc',
      'color': [201, 0, 0],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'lat0': 'source_lat',
       'lng0': 'source_lng',
       'lat1': 'target_lat',
       'lng1': 'target_lng'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'thickness': 6.7,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'sizeRange': [0, 10],
       'targetColor': [155, 165, 46]},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear'}}],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'bahnhof': [{'name': 'EVA_NR',
        'format': None},
       {'name': 'DS100', 'format': None},
       {'name': 'NAME', 'format': None},
       {'name': 'Verkehr', 'format': None},
       {'name': 'state', 'format': None}],
      'route': [{'name': 'cumulative_distance_km', 'format': None},
       {'name': 'cumulative_energy_kwh', 'format': None},
       {'name': 'cumulative_min', 'format': None},
       {'name': 'between_distance_km', 'format': None},
       {'name': 'between_energy_kwh', 'format': None},
       {'name': 'between_min', 'format': None}]},
     'compareMode': False,
     'compareType': 'absolute',
     'enabled': True},
    'brush': {'size': 0.5, 'enabled': False},
    'geocoder': {'enabled': False},
    'coordinate': {'enabled': False}},
   'layerBlending': 'normal',
   'splitMaps': [],
   'animationConfig': {'currentTime': None, 'speed': 1}},
  'mapState': {'bearing': 0,
   'dragRotate': False,
   'latitude': 50.229970311985326,
   'longitude': 8.763317706068714,
   'pitch': 0,
   'zoom': 5.056382361569942,
   'isSplit': False}}}

In [77]:
map_1 = KeplerGl(height=800, data={"bahnhof": bahnhof, "route": route_df}, config=config)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [78]:
from IPython.display import Javascript
display(Javascript('''
  google.colab.widgets.installCustomManager('https://ssl.gstatic.com/colaboratory-static/widgets/colab-cdn-widget-manager/6a14374f468a145a/manager.min.js');
'''))

map_1

<IPython.core.display.Javascript object>

KeplerGl(config={'version': 'v1', 'config': {'visState': {'layers': [{'type': 'point', 'config': {'dataId': 'b…

In [68]:
map_1.config

{'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': '46gx0mf',
     'type': 'point',
     'config': {'dataId': 'bahnhof',
      'label': 'Point',
      'color': [18, 92, 119],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'lat': 'latitude', 'lng': 'longitude', 'altitude': None},
      'isVisible': True,
      'visConfig': {'radius': 10,
       'fixedRadius': False,
       'opacity': 0.09,
       'outline': False,
       'thickness': 2,
       'strokeColor': None,
       'colorRange': {'name': 'Uber Viz Qualitative 4',
        'type': 'qualitative',
        'category': 'Uber',
        'colors': ['#12939A',
         '#DDB27C',
         '#88572C',
         '#FF991F',
         '#F15C17',
         '#223F9A',
         '#DA70BF',
         '#125C77',
         '#4DC19C',
         '#776E57',
         '#17B8BE',
         '#F6D18A',
         '#B7885E',
         '#FFCB99',
         '#F89570',
         '#829AE3',
         '#E79FD5',
         '#1E96BE',
   